<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 2: Working with Text
# 第二章：文本处理

Packages that are being used in this notebook:
本章节使用的包：

In [1]:
# 从importlib.metadata导入version函数
from importlib.metadata import version

# 打印torch包的版本号
print("torch version:", version("torch"))
# 打印tiktoken包的版本号 
print("tiktoken version:", version("tiktoken"))



torch version: 2.5.0
tiktoken version: 0.8.0


- This chapter covers data preparation and sampling to get input data "ready" for the LLM
- 本章涵盖了数据准备和采样,以使输入数据为LLM做好"准备"

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

 ## 2.1 Understanding word embeddings
 ## 2.1 理解词嵌入

 - No code in this section
 - 本节没有代码

 - There are many forms of embeddings; we focus on text embeddings in this book
 - 嵌入有很多形式;在本书中我们专注于文本嵌入

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

 - LLMs work with embeddings in high-dimensional spaces (i.e., thousands of dimensions)
  - LLM在高维空间中使用嵌入(即数千个维度)
 - Since we can't visualize such high-dimensional spaces (we humans think in 1, 2, or 3 dimensions), the figure below illustrates a 2-dimensional embedding space
 - 由于我们无法可视化如此高维的空间(我们人类只能思考1、2或3维),下图展示了一个二维嵌入空间

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

# ## 2.2 Tokenizing text
# ## 2.2 文本分词

 - In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters
- 在本节中,我们对文本进行分词,这意味着将文本分解成更小的单位,如单个词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

 - Load raw text we want to work with
 - 加载我们要处理的原始文本
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story
 - [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 是一个公共领域的短篇小说

In [2]:
# 导入os模块用于文件和目录操作
import os
# 导入urllib.request模块用于从网络下载文件
import urllib.request

# 检查文件是否存在
if not os.path.exists("the-verdict.txt"):
    # 设置文件的URL地址
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    # 设置本地保存的文件路径
    file_path = "the-verdict.txt"
    # 从URL下载文件并保存到本地
    urllib.request.urlretrieve(url, file_path)

 - (If you encounter an `ssl.SSLCertVerificationError` when executing the previous code cell, it might be due to using an outdated Python version; you can find [more information here on GitHub](https://github.com/rasbt/LLMs-from-scratch/pull/403))
 - (如果在执行前面的代码单元格时遇到 `ssl.SSLCertVerificationError` 错误,这可能是由于使用了过时的Python版本;你可以在[GitHub上找到更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403))

In [3]:
# 以utf-8编码打开文件
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    # 读取文件内容到raw_text变量
    raw_text = f.read()
    
# 打印文本总字符数
print("Total number of character:", len(raw_text))
# 打印文本的前99个字符
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


 - The goal is to tokenize and embed this text for an LLM
 - 目标是对这段文本进行分词并嵌入到LLM中
 - Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above  
 - 让我们基于一些简单的示例文本开发一个简单的分词器,然后将其应用到上面的文本中
 - The following regular expression will split on whitespaces
 - 下面的正则表达式将在空格处进行分割

In [4]:
# 导入正则表达式模块
import re

# 定义一个示例文本字符串
text = "Hello, world. This, is a test."
# 使用正则表达式按空格分割文本,保留分隔符
result = re.split(r'(\s)', text)

# 打印分割结果
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


 - We don't only want to split on whitespaces but also commas and periods, so let's modify the regular expression to do that as well
 - 我们不仅要在空格处分割,还要在逗号和句号处分割,所以让我们修改正则表达式来实现这一点

In [5]:
# 使用正则表达式按逗号、句号和空格分割文本,保留分隔符
result = re.split(r'([,.]|\s)', text)

# 打印分割结果
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


 - As we can see, this creates empty strings, let's remove them
 - 正如我们所看到的,这会创建空字符串,让我们把它们删除

In [6]:
# 从每个项目中去除空白字符
# 使用列表推导式过滤掉空字符串
# 将结果存储在result变量中
result = [item for item in result if item.strip()]
# 打印处理后的结果
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


 - This looks pretty good, but let's also handle other types of punctuation, such as periods, question marks, and so on
 - 这看起来不错,但让我们也处理其他类型的标点符号,比如句号、问号等

In [7]:
# 定义一个包含标点符号和特殊字符的示例文本
text = "Hello, world. Is this-- a test?"

# 使用正则表达式按标点符号、破折号和空格分割文本,保留分隔符
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
# 从每个项目中去除空白字符并过滤掉空字符串
result = [item.strip() for item in result if item.strip()]
# 打印分割结果
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


 - This is pretty good, and we are now ready to apply this tokenization to the raw text
 - 这个效果很好,现在我们可以将这个分词方法应用到原始文本了

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
# 使用正则表达式按标点符号、破折号和空格分割原始文本,保留分隔符
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# 从每个项目中去除空白字符并过滤掉空字符串
preprocessed = [item.strip() for item in preprocessed if item.strip()]
# 打印前30个分词结果
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


 - Let's calculate the total number of tokens
 - 让我们计算总的词元数量

In [9]:
# 打印词元数量
print(len(preprocessed))

4690


 ## 2.3 Converting tokens into token IDs
 ## 2.3 将词元转换为词元ID

 - Next, we convert the text tokens into token IDs that we can process via embedding layers later
 - 接下来,我们将文本词元转换为词元ID,以便之后通过嵌入层进行处理

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

 - From these tokens, we can now build a vocabulary that consists of all the unique tokens
 - 从这些词元中,我们现在可以构建一个包含所有唯一词元的词汇表

In [10]:
# 对预处理后的词元列表去重并排序,得到所有唯一词元
all_words = sorted(set(preprocessed))
# 计算词汇表大小
vocab_size = len(all_words)

# 打印词汇表大小
print(vocab_size)

1130


In [11]:
# 创建词汇表字典,将每个词元映射到一个唯一的整数ID
# token作为键,integer作为值,使用enumerate生成整数序列
vocab = {token:integer for integer,token in enumerate(all_words)}

 - Below are the first 50 entries in this vocabulary:
 - 以下是词汇表中的前50个条目:

In [12]:
# 遍历词汇表中的条目
for i, item in enumerate(vocab.items()):
    # 打印每个词元-ID对
    print(item)
    # 只打印前50个条目
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


 - Below, we illustrate the tokenization of a short sample text using a small vocabulary:
 - 下面,我们使用一个小型词汇表来演示对短文本进行分词的过程:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

 - Putting it now all together into a tokenizer class
 - 现在将所有内容整合到一个分词器类中

In [13]:
# 分词器类
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # 存储词汇表字典,将字符串映射到整数ID
        self.str_to_int = vocab
        # 创建反向映射字典,将整数ID映射回字符串
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        # 使用正则表达式分割文本,保留标点符号和空格
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        # 去除空白并过滤掉空字符串
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # 将每个词元转换为对应的整数ID
        ids = [self.str_to_int[s] for s in preprocessed]
        # 返回ids
        return ids
        
    def decode(self, ids):
        # 将整数ID序列转换回文本,用空格连接
        text = " ".join([self.int_to_str[i] for i in ids])
        # 删除标点符号前的空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        # 返回text
        return text

 - The `encode` function turns text into token IDs
 - The `decode` function turns token IDs back into text
 - `encode` 函数将文本转换为词元ID
 - `decode` 函数将词元ID转换回文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

 - We can use the tokenizer to encode (that is, tokenize) texts into integers
 - These integers can then be embedded (later) as input of/for the LLM
 - 我们可以使用分词器将文本编码(即分词)为整数
 - 这些整数随后可以被嵌入(后面会讲到)作为LLM的输入

In [14]:
# 创建一个SimpleTokenizerV1实例,使用vocab词汇表
tokenizer = SimpleTokenizerV1(vocab)

# 定义要编码的文本字符串
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
# 使用tokenizer将文本编码为整数ID序列
ids = tokenizer.encode(text)
# 打印编码后的整数ID序列
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


 - We can decode the integers back into text
 - 我们可以将整数解码回文本

In [15]:
# 使用tokenizer将整数ID序列解码回文本
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
# 先使用tokenizer.encode()将文本编码为整数ID序列
# 然后使用tokenizer.decode()将整数ID序列解码回文本
# 这个操作相当于对文本进行了一次编码和解码的往返转换
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

 ## 2.4 Adding special context tokens
 ## 2.4 添加特殊上下文标记

 - It's useful to add some "special" tokens for unknown words and to denote the end of a text
 - 为未知词和标记文本结尾添加一些"特殊"标记是很有用的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

 - Some tokenizers use special tokens to help the LLM with additional context
 - 一些分词器使用特殊标记来帮助LLM获取额外的上下文信息
 - Some of these special tokens are
 - 这些特殊标记包括:
   - `[BOS]` (beginning of sequence) marks the beginning of text
   - `[BOS]` (序列开始)标记文本的开始
   - `[EOS]` (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
   - `[EOS]` (序列结束)标记文本的结束(通常用于连接多个不相关的文本,例如两篇不同的维基百科文章或两本不同的书等)
   - `[PAD]` (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
   - `[PAD]` (填充)如果我们使用大于1的批量大小训练LLM(我们可能包含多个不同长度的文本;使用填充标记,我们将较短的文本填充到最长长度,使所有文本长度相等)
 - `[UNK]` to represent words that are not included in the vocabulary
 - `[UNK]` 用于表示词汇表中不包含的词
 
 - Note that GPT-2 does not need any of these tokens mentioned above but only uses an `<|endoftext|>` token to reduce complexity
 - 注意,GPT-2不需要上述任何标记,而只使用`<|endoftext|>`标记来降低复杂性
 - The `<|endoftext|>` is analogous to the `[EOS]` token mentioned above
 - `<|endoftext|>`类似于上面提到的`[EOS]`标记
 - GPT also uses the `<|endoftext|>` for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)
 - GPT也使用`<|endoftext|>`进行填充(因为我们在训练批量输入时通常使用掩码,所以无论如何都不会关注填充标记,因此这些标记是什么并不重要)
 - GPT-2 does not use an `<UNK>` token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units which we will discuss in a later section
 - GPT-2不使用`<UNK>`标记来表示词汇表外的词;相反,GPT-2使用字节对编码(BPE)分词器,将词分解为子词单元,我们将在后面的章节中讨论这一点



 - We use the `<|endoftext|>` tokens between two independent sources of text:
 - 我们在两个独立的文本源之间使用`<|endoftext|>`标记:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

 - Let's see what happens if we tokenize the following text:
 - 让我们看看对以下文本进行分词会发生什么:

In [17]:
# 创建一个SimpleTokenizerV1实例,使用vocab词汇表
tokenizer = SimpleTokenizerV1(vocab)

# 定义要分词的测试文本
text = "Hello, do you like tea. Is this-- a test?"

# 使用tokenizer对文本进行编码(转换为token ID)
# tokenizer.encode(text)

 - The above produces an error because the word "Hello" is not contained in the vocabulary
 - 上面产生错误是因为词汇表中不包含"Hello"这个词
 - To deal with such cases, we can add special tokens like `"<|unk|>"` to the vocabulary to represent unknown words
 - 为了处理这种情况,我们可以在词汇表中添加特殊标记如`"<|unk|>"`来表示未知词
 - Since we are already extending the vocabulary, let's add another token called `"<|endoftext|>"` which is used in GPT-2 training to denote the end of a text (and it's also used between concatenated text, like if our training datasets consists of multiple articles, books, etc.)
 - 既然我们已经在扩展词汇表,让我们再添加一个叫做`"<|endoftext|>"`的标记,它在GPT-2训练中用于表示文本的结束(它也用于连接的文本之间,比如当我们的训练数据集包含多篇文章、书籍等)

In [18]:
# 对预处理后的token列表去重并排序
all_tokens = sorted(list(set(preprocessed)))
# 添加特殊标记 <|endoftext|> 和 <|unk|> 到token列表
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

# 创建词汇表字典,将token映射到整数ID
vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
# 获取词汇表的大小
len(vocab.items())

1132

In [20]:
# 遍历词汇表中最后5个项目并打印
# 使用enumerate获取索引和键值对
# list(vocab.items())[-5:] 获取词汇表中最后5个项目
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


 - We also need to adjust the tokenizer accordingly so that it knows when and how to use the new `<unk>` token
 - 我们还需要相应地调整分词器,使其知道何时以及如何使用新的`<unk>`标记

In [21]:
# 分词器V2版本
class SimpleTokenizerV2:
    # 初始化函数,接收词汇表作为参数
    def __init__(self, vocab):
        # 将词汇表存储为字符串到整数的映射
        self.str_to_int = vocab
        # 创建整数到字符串的反向映射
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    # 编码函数,将文本转换为token ID
    def encode(self, text):
        # 使用正则表达式分割文本,保留标点符号
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # 去除空白并过滤掉空字符串
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # 将不在词汇表中的token替换为<|unk|>
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        # 将token转换为对应的整数ID
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    # 解码函数,将token ID转换回文本
    def decode(self, ids):
        # 将ID转换为对应的token并用空格连接
        text = " ".join([self.int_to_str[i] for i in ids])
        # 删除标点符号前的空格
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

 Let's try to tokenize text with the modified tokenizer:
 让我们用修改后的分词器来尝试分词:

In [22]:
# 创建SimpleTokenizerV2的实例,使用vocab作为词汇表
tokenizer = SimpleTokenizerV2(vocab)

# 定义第一个测试文本
text1 = "Hello, do you like tea?"
# 定义第二个测试文本
text2 = "In the sunlit terraces of the palace."

# 使用<|endoftext|>标记将两个文本连接起来
text = " <|endoftext|> ".join((text1, text2))

# 打印连接后的文本
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
# 使用分词器对文本进行编码,将返回token ID列表
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
# 先对文本进行编码得到token ID列表,再将token ID列表解码回文本
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

 - GPT-2 used BytePair encoding (BPE) as its tokenizer
 - GPT-2使用字节对编码(BPE)作为其分词器

 - it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
 - 它允许模型将不在预定义词汇表中的单词分解成更小的子词单元甚至单个字符,从而能够处理词汇表外的单词

 - For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
 - 例如,如果GPT-2的词汇表中没有"unfamiliarword"这个词,它可能会根据训练好的BPE合并规则将其分词为["unfam", "iliar", "word"]或其他子词组合

 - The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
 - 原始的BPE分词器可以在这里找到: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)

 - In this chapter, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
 - 在本章中,我们使用OpenAI开源的[tiktoken](https://github.com/openai/tiktoken)库中的BPE分词器,该库使用Rust实现其核心算法以提高计算性能

 - I created a notebook in the [./bytepair_encoder](../02_bonus_bytepair-encoder) that compares these two implementations side-by-side (tiktoken was about 5x faster on the sample text)
 - 我在[./bytepair_encoder](../02_bonus_bytepair-encoder)中创建了一个notebook,对这两种实现进行了并排比较(在样本文本上tiktoken快了约5倍)

In [25]:
# 安装tiktoken库
# pip install tiktoken

In [26]:
# 导入importlib模块用于获取包版本信息
import importlib
# 导入tiktoken分词器库
import tiktoken

# 打印tiktoken库的版本号
print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [27]:
# 使用tiktoken库获取GPT-2的分词器
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
# 定义一个包含特殊标记<|endoftext|>的测试文本
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

# 使用tokenizer将文本编码为整数序列
# allowed_special参数指定允许的特殊标记
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

# 打印编码后的整数序列
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
# 使用tokenizer将整数序列解码回文本字符串
strings = tokenizer.decode(integers)

# 打印解码后的文本
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


 - BPE tokenizers break down unknown words into subwords and individual characters:
 - BPE分词器会将未知词分解成子词和单个字符:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 Data sampling with a sliding window
## 2.6 使用滑动窗口进行数据采样

 - We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:
 - 我们训练大语言模型时是一次生成一个词,所以我们需要相应地准备训练数据,其中序列中的下一个词代表要预测的目标:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
# 以utf-8编码打开文件"the-verdict.txt"进行读取
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 使用tokenizer对原始文本进行编码
enc_text = tokenizer.encode(raw_text)
# 打印编码后的文本长度
print(len(enc_text))

5145


 - For each text chunk, we want the inputs and targets
- 对于每个文本块,我们需要输入和目标
 - Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right  
- 由于我们希望模型预测下一个词,目标是将输入向右移动一个位置

In [31]:
# 从第50个位置开始截取编码后的文本作为样本
enc_sample = enc_text[50:]

In [32]:
# 设置上下文窗口大小为4
context_size = 4

# 从enc_sample中获取前context_size个token作为输入x
x = enc_sample[:context_size]
# 从enc_sample中获取第2个到第context_size+1个token作为目标y
y = enc_sample[1:context_size+1]

# 打印输入x和目标y
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


 - One by one, the prediction would look like as follows:
 - 逐个来看,预测过程如下:

In [33]:
# 遍历从1到context_size的范围
for i in range(1, context_size+1):
    # 获取从开始到第i个位置的上下文序列
    context = enc_sample[:i]
    # 获取第i个位置的目标token
    desired = enc_sample[i]

    # 打印上下文序列和目标token
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
# 遍历从1到context_size的范围
for i in range(1, context_size+1):
    # 获取从开始到第i个位置的上下文序列
    context = enc_sample[:i]
    # 获取第i个位置的目标token
    desired = enc_sample[i]

    # 将token ID解码回文本并打印上下文和目标
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


 - We will take care of the next-word prediction in a later chapter after we covered the attention mechanism
 - 我们将在介绍注意力机制后的章节中处理下一个词的预测
 - For now, we implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one
 - 现在,我们实现一个简单的数据加载器,它遍历输入数据集并返回错位一位的输入和目标

 - Install and import PyTorch (see Appendix A for installation tips)
 - 安装并导入PyTorch(安装提示请参考附录A)

In [35]:
# 导入PyTorch库
import torch
# 打印PyTorch版本号
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.0+cpu


 - We use a sliding window approach, changing the position by +1:
 - 我们使用滑动窗口方法,每次将位置向前移动1:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

 - Create dataset and dataloader that extract chunks from the input text dataset
 - 创建数据集和数据加载器,从输入文本数据集中提取文本块

In [36]:
# 从torch.utils.data导入Dataset和DataLoader类
from torch.utils.data import Dataset, DataLoader


# 定义GPTDatasetV1类,继承自Dataset
class GPTDatasetV1(Dataset):
    # 初始化函数,接收文本、分词器、最大长度和步长作为参数
    def __init__(self, txt, tokenizer, max_length, stride):
        # 初始化输入ID列表
        self.input_ids = []
        # 初始化目标ID列表 
        self.target_ids = []

        # 使用分词器对整个文本进行编码
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 使用滑动窗口将文本分割成重叠的序列,每个序列长度为max_length
        for i in range(0, len(token_ids) - max_length, stride):
            # 获取输入序列片段
            input_chunk = token_ids[i:i + max_length]
            # 获取目标序列片段(比输入序列向后移动一位)
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # 将输入序列转换为tensor(张量)并添加到input_ids列表
            # tensor是PyTorch中的多维数组数据结构,类似numpy的ndarray,用于存储和计算深度学习中的数据
            self.input_ids.append(torch.tensor(input_chunk))
            # 将目标序列转换为tensor(张量)并添加到target_ids列表
            self.target_ids.append(torch.tensor(target_chunk))

    # 返回数据集长度
    def __len__(self):
        return len(self.input_ids)

    # 根据索引返回一个数据样本
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    创建数据加载器的函数
    
    参数:
        txt: 输入的原始文本
        batch_size: 每个批次的样本数,默认为4
        max_length: 每个序列的最大长度,默认为256
        stride: 滑动窗口的步长,默认为128
        shuffle: 是否打乱数据顺序,默认为True
        drop_last: 是否丢弃最后不完整的批次,默认为True
        num_workers: 用于数据加载的子进程数,默认为0
        
    返回:
        dataloader: PyTorch的DataLoader对象
    """
    
    # 初始化GPT-2分词器
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集实例
    # txt: 输入的原始文本
    # tokenizer: 分词器对象
    # max_length: 每个序列的最大长度
    # stride: 滑动窗口的步长
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建并返回数据加载器
    # dataset: 上面创建的数据集实例
    # batch_size: 每个批次的样本数
    # shuffle: 是否打乱数据顺序
    # drop_last: 是否丢弃最后不完整的批次
    # num_workers: 用于数据加载的子进程数
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

 - Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:
 - 让我们用batch size=1测试数据加载器,用于上下文长度为4的LLM:

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


 - An example using stride equal to the context length (here: 4) as shown below:
 - 一个使用步长等于上下文长度(这里是4)的例子,如下所示:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

 - We can also create batched outputs
 - Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting
 - 我们也可以创建批量输出
 - 注意这里我们增加了步长以避免批次之间的重叠,因为更多的重叠可能导致过拟合

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


 ## 2.7 Creating token embeddings
 ## 2.7 创建词元嵌入

 - The data is already almost ready for an LLM
 - 数据已经几乎准备好用于LLM了
 - But lastly let us embed the tokens in a continuous vector representation using an embedding layer  
 - 但最后让我们使用嵌入层将词元嵌入到连续的向量表示中
 - Usually, these embedding layers are part of the LLM itself and are updated (trained) during model training
 - 通常,这些嵌入层是LLM本身的一部分,并在模型训练期间进行更新(训练)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

 - Suppose we have the following four input examples with input ids 2, 3, 5, and 1 (after tokenization):
 - 假设我们有以下四个输入示例,它们的输入ID分别是2、3、5和1(经过分词后):

In [42]:
# 创建一个包含4个输入ID的张量
# 这些ID代表了经过分词后的4个不同的词元
input_ids = torch.tensor([2, 3, 5, 1])

 - For the sake of simplicity, suppose we have a small vocabulary of only 6 words and we want to create embeddings of size 3:
 - 为了简单起见,假设我们有一个只包含6个词的小词汇表,并且我们想创建大小为3的嵌入:

In [43]:
# 词汇表大小为6个词
vocab_size = 6
# 嵌入向量维度为3
output_dim = 3

# 设置随机种子以确保结果可复现
torch.manual_seed(123)
# 创建一个嵌入层,将词汇表中的词映射到3维向量空间
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- This would result in a 6x3 weight matrix:

In [44]:
# 打印嵌入层的权重矩阵
# 这是一个6x3的矩阵,每一行代表词汇表中一个词的3维嵌入向量
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- For those who are familiar with one-hot encoding, the embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully-connected layer, which is described in the supplementary code in [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)
 - 对于熟悉独热编码的人来说,上述嵌入层方法本质上只是实现独热编码后接全连接层矩阵乘法的一种更高效方式,这在补充代码[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)中有详细描述
 - Because the embedding layer is just a more efficient implementation that is equivalent to the one-hot encoding and matrix-multiplication approach it can be seen as a neural network layer that can be optimized via backpropagation
 - 由于嵌入层只是独热编码和矩阵乘法方法的一种更高效实现,它可以被视为一个可以通过反向传播优化的神经网络层

 - To convert a token with id 3 into a 3-dimensional vector, we do the following:
 - 要将ID为3的词元转换为3维向量,我们执行以下操作:

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


 - Note that the above is the 4th row in the `embedding_layer` weight matrix
 - 注意上面的结果是`embedding_layer`权重矩阵的第4行
 - To embed all four `input_ids` values above, we do
 - 要嵌入上面的所有四个`input_ids`值,我们执行以下操作

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


 - An embedding layer is essentially a look-up operation:
 - 嵌入层本质上是一个查找操作:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

 - **You may be interested in the bonus content comparing embedding layers with regular linear layers: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**
 - **你可能对比较嵌入层和常规线性层的补充内容感兴趣:[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

 ## 2.8 Encoding word positions
 ## 2.8 编码词位置

 - Embedding layer convert IDs into identical vector representations regardless of where they are located in the input sequence:
 - 嵌入层将ID转换为相同的向量表示,无论它们在输入序列中的位置如何:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

 - Positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model:
 - 位置嵌入与词元嵌入向量相结合,形成大型语言模型的输入嵌入:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

 - The BytePair encoder has a vocabulary size of 50,257:
 - BytePair编码器的词汇表大小为50257:
 - Suppose we want to encode the input tokens into a 256-dimensional vector representation:
 - 假设我们要将输入词元编码为256维的向量表示:

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

 - If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
 - If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:
 - 如果我们从数据加载器中采样数据,我们将每个批次中的词元嵌入到256维向量中
 - 如果我们有一个批次大小为8,每个序列包含4个词元,这将得到一个8 x 4 x 256的张量:

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


 - GPT-2 uses absolute position embeddings, so we just create another embedding layer:
 - GPT-2使用绝对位置嵌入,所以我们只需创建另一个嵌入层:

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


 - To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:
 - 要创建在LLM中使用的输入嵌入,我们只需将词元嵌入和位置嵌入相加:

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


 - In the initial phase of the input processing workflow, the input text is segmented into separate tokens
 - Following this segmentation, these tokens are transformed into token IDs based on a predefined vocabulary:
 - 在输入处理工作流的初始阶段,输入文本被分割成单独的词元
 - 在分割之后,这些词元根据预定义的词汇表被转换成词元ID:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

 # Summary and takeaways
 # 总结和要点

 See the [./dataloader.ipynb](./dataloader.ipynb) code notebook, which is a concise version of the data loader that we implemented in this chapter and will need for training the GPT model in upcoming chapters.
 
 请查看[./dataloader.ipynb](./dataloader.ipynb)代码笔记本,这是我们在本章实现的数据加载器的简洁版本,在接下来的章节中训练GPT模型时会用到它。
 
 See [./exercise-solutions.ipynb](./exercise-solutions.ipynb) for the exercise solutions.
 
 练习题解答请查看[./exercise-solutions.ipynb](./exercise-solutions.ipynb)。